<font size="1">Copyright 2021, by the California Institute of Technology. ALL RIGHTS RESERVED. United States Government sponsorship acknowledged. Any commercial use must be negotiated with the Office of Technology Transfer at the California Institute of Technology.</font>
    
<font size="1">This software may be subject to U.S. export control laws and regulations. By accepting this document, the user agrees to comply with all applicable U.S. export laws and regulations. User has the responsibility to obtain export licenses, or other export authority as may be required, before exporting such information to foreign countries or providing access to foreign persons.<font>

## User-local Environment Creation
**This notebook must be run in the base kernel.**

This notebook provides the means by which additional conda environments that support specific science data packages are created local to the user (i.e.'user-local'). The implication of user-local installations is that they are retained across logins and server restarts (since the ~jovyan directory is retained and remounted each time). It also permits the user to customize the environment, again with those changes persisted across sessions.

This notebook provides the steps to create the following user-local environments:
- ISCE2/ISCE3
- MintPy/ARIATools
- PLAnT

Each environment also contains the PCM APIs (pele-client, otello, osaka) and tools (notebook-pge-wrapper).

#### Install kernda
Kernda will be used to make ipython aware of the environments we create.

In [6]:
pip install kernda

Note: you may need to restart the kernel to use updated packages.


### What environments are currently available?
Come back and re-execute this cell any time to see what has and hasn't been created. Those found under /home/jovyan are user-local while others are global.

In [1]:
%conda env list

# conda environments:
#
isce                     /home/jovyan/.local/envs/isce
isce_raider              /home/jovyan/.local/envs/isce_raider
mintpy                   /home/jovyan/.local/envs/mintpy
base                     /opt/conda
pcm_tools                /opt/conda/envs/pcm_tools


Note: you may need to restart the kernel to use updated packages.


### Configure conda
This will make conda recognize directories under ~/.local/envs as their environment name. This only needs to be run once, ever.

In [2]:
%%bash
mkdir -p ~/.local/envs
conda config --append envs_dirs ~/.local/envs

### ISCE2 + Solid Earth ATBDs


In [ ]:
%%bash

env_base="/home/jovyan/.local/envs/solid_earth_atbd"

# (re)create the ISCE2 environment that includes the packages required for Solid Earth ATBDs
conda env create -f env_files/isce_conda_env.yml -p ${env_base} --force
# for now, have to use a slightly customized ATBD dependency yml file which specifies the correct version of dem_stitcher
conda env update -f env_files/solid_earth_atbd.yml -p ${env_base}

# clone the Solid Earth ATBD repo into home directory
pushd ~
git clone 
cd 

# update the ipykernel to recognize the new kernel
conda run -n solid_earth_atbd python -m ipykernel install --user --name solid_earth_atbd --display-name "solid_earth_atbd"
# now invoke kernda to modify the kernel file to ensure the kernel is invoked from within the conda environment
conda run -n solid_earth_atbd kernda --env-dir ${env_base} --display-name solid_earth_atbd -o ~/.local/share/jupyter/kernels/solid_earth_atbd/kernel.json

isce_package=${env_base}/lib/python3.10/site-packages/isce

# install the license
popd
tar zxf support_files/stanford_components.tgz -C ${isce_package}

# Additional ISCE set-up
source /opt/conda/bin/activate ${env_base}
conda env config vars set -n solid_earth_atbd PATH=${env_base}/bin:${isce_package}/applications:${PATH}
conda env config vars set -n solid_earth_atbd PYTHONPATH=${isce_package}:${PYTHONPATH}

# Additional Solid Earth ATBD set-up
conda env config vars set -n solid_earth_atbd PYTHONPATH=${isce_package}:${PYTHONPATH}


Solving environment: ...working... done




==> WARNING: A newer version of conda exists. <==
  current version: 4.14.0
  latest version: 22.9.0

Please update conda by running

    $ conda update -n base -c conda-forge conda





imagecodecs-2022.9.2 | 2.7 MB    | ########## | 100% 
setuptools-65.4.1    | 776 KB    | ########## | 100% 
ply-3.11             | 44 KB     | ########## | 100% 
soupsieve-2.3.2.post | 34 KB     | ########## | 100% 
libstdcxx-ng-12.1.0  | 4.3 MB    | ########## | 100% 
brunsli-0.1          | 200 KB    | ########## | 100% 
scons-4.4.0          | 2.5 MB    | ########## | 100% 
ld_impl_linux-64-2.3 | 667 KB    | ########## | 100% 
zlib-1.2.12          | 92 KB     | ########## | 100% 
python_abi-3.10      | 4 KB      | ########## | 100% 
numpy-1.23.3         | 7.0 MB    | ########## | 100% 
libnetcdf-4.8.1      | 1.5 MB    | ########## | 100% 
libkml-1.3.0         | 610 KB    | ########## | 100% 
libxkbcommon-1.0.3   | 581 KB    | ########## | 100% 
pandocfilters-1.5.0  | 11 KB     | ########## | 100% 
libffi-3.4.2         | 57 KB     | ########## | 100% 
h5py-3.7.0           | 1.3 MB    | ########## | 100% 
python-3.10.6        | 29.0 MB   | ########## | 100% 
cffi-1.15.1          | 231 

### Creating other user-local environments.

As shown above, the command used to create a conda environment is:

```conda env create <arguments>```

In the above cases, the ```-f``` switch is used to denote the input file which describes the environment's contents, including the source conda channels (e.g. default, conda-forge) and the environment prefix (in this case, subdirectories of /home/jovyan/.local/envs). The prefix is responsible for designating where the environment's contents will reside and if a subdirectory of the user (i.e. jovyan), the environment is user-local. Since the ~jovyan directory is persisted and remounted for each jupyter login, these environments also persist and do not have to be created each time you log into jupyter.

The --force flag in the command lines used above forces conda to recreate (rather than just update) the environment from scratch.

For clarity and organization, it's best to maintain the local environments in the same user subdirectory.

To create a new user-local environment from scratch, use the following command:

```conda env create -p /home/jovyan/.local/envs/<env-name>```


To start with a copy of another environment (such as pcm_tools), omit the ```env``` and use the ```--clone``` command line option:

```conda create -p /home/jovyan/.local/envs/<env-name> --clone pcm_tools```

Once created, switch to the new environment accordingly (this is only applicable in terminal windows):

```conda activate <env-name>```

Currently active environments and their prefix can be listed by:

```conda env list```

An environment can be removed by:

```conda env remove -n <env name>``` - to remove by name

or

```conda env remove -p <prefix>``` - to remove by prefix

Finally, to make ipython aware of the environment, invoke ipykernel:

```conda run -n <env name> python -m ipykernel install --user --name <env name> --display-name <env name>```

then run kernda:

```conda run -n <env name> kernda --env-dir ~/.local/share/<env name> --display-name <env name> \
         -o ~/.local/share/jupyter/kernels/<env name>/kernel.json```

Running kernda ensures that the kernel is launched from within the associated conda environment. This means that any shell commands invoked within the notebook (e.g. using the %%bash magic) execute within that environment.

You will need to reload the jupyter landing page in order to see the environment under the ```New``` pull down.

<font size="1">This notebook is compatible with NISAR Jupyter Server Image v1.7.3 and above</font>